### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [28]:
###SEMPRE que remover a VectorDB, é necessário reiniciar o Kernel; do contrário tomaremos erro. 
import shutil

shutil.rmtree("InnovationCamp_VectorDB")

In [1]:
from langchain.document_loaders import DirectoryLoader, JSONLoader

# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["source"] = record.get("metadata")['source']

    return metadata

loader = DirectoryLoader('./JSON/', glob='**/*.json', 
                         show_progress=True, 
                         loader_cls=JSONLoader,
                         loader_kwargs = {'jq_schema': '.', 'content_key' : "page_content", 'metadata_func' : metadata_func}
                         )

data = loader.load()

 50%|█████     | 4/8 [00:00<00:00, 350.33it/s]


In [2]:
data[1].page_content

'A mamografia utiliza um aparelho (mamógrafo) desenvolvido apenas para observar o tecido mamário. A máquina obtém raios-x em doses mais baixas do que os raios-x normais. Como eles não atravessam o tecido com facilidade, a máquina possui 2 placas que comprimem a mama para separar o tecido. Isso gera uma imagem melhor e permite que menos radiação seja usada. Como a mamografia é realizada? A mulher é posicionada em pé, de forma que a mama fique entre as duas placas do mamógrafo, que é onde as imagens serão capturadas. É um procedimento simples e o desconforto pela compressão das mamas é passageiro. As imagens são obtidas, normalmente em 2 ângulos diferentes para cada mama. Imagens adicionais podem ser requeridas para avaliar melhor algum achado específico. Quais são os tipos de mamografia? Existem dois tipos: a convencional, realizada com um aparelho analógico, e a digital, realizada com um mamógrafo digital. Há ainda a tomossíntese ou mamografia 3D, que surgiu mais recentemente e nada ma

In [3]:
data[0].metadata['source']

'https://grupooncoclinicas.com/tudo-sobre-o-cancer/tratamentos-para-o-cancer/radioterapia'

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
all_splits = text_splitter.split_documents(data)   

In [5]:
len(all_splits)

29

In [6]:
all_splits[1].page_content

'afetadas durante a radioterapia, mas de forma menos intensa. Embora elas tenham uma maior capacidade de recuperação, em alguns tratamentos as sessões são fracionadas para dar tempo de o organismo se fortalecer novamente. Além disso, as doses de radiação e o tempo de aplicação são calculados de acordo com o tipo e o tamanho do tumor, o que também auxilia na preservação das células saudáveis. Segundo a OMS (Organização Mundial de Saúde), aproximadamente 60% dos pacientes com diagnóstico de câncer'

In [7]:
all_splits[0].metadata['source']

'https://grupooncoclinicas.com/tudo-sobre-o-cancer/tratamentos-para-o-cancer/radioterapia'

In [8]:
from langchain.vectorstores import Chroma
from langchain.embeddings import CohereEmbeddings


#Para gerar uma chave API Cohere, basta entrar em https://dashboard.cohere.com/api-keys; 
#Há tanto uma API Trial (gratuita e mais limitada) quanto uma API Produtiva

cohere = CohereEmbeddings(cohere_api_key=cohere_api_key, model = "multilingual-22-12")

#Esta etapa demora um pouco (5-8 min), mas podemos gravar a VectorStore no disco e apenas consumir daqui em diante
vectorstore = Chroma.from_documents(documents=all_splits, embedding=cohere)

As próximas duas células possuem o código utilizado para criar a VectorStore e publicá-la no disco da máquina. Dessa forma, não é necessário gerar esta VectorStore todas as vezes que formos consumi-la, basta carregar da memória. Basta descomentar as linhas com ##.

In [9]:
#Nome da pasta para onde vai a VectorStore
persist_directory = 'InnovationCamp_VectorDB'

vectordb = Chroma.from_documents(documents=all_splits,                               
                                 # Chose the embedding you want to use
                                 embedding=cohere,
                                 persist_directory=persist_directory)

# Persistir a VectorStore para o disco
vectorstore.persist()

In [10]:
#Nome da pasta de onde vamos carregar a VectorStore
persist_directory = 'InnovationCamp_VectorDB'

#Carregando a VectorStore do disco
vectorstore = Chroma(persist_directory=persist_directory,
                  embedding_function=cohere)

In [11]:
question = "Radioerapia causa quais efeitos colaterais?"
docs = vectorstore.similarity_search(question, k=6)

In [12]:
docs[0].page_content

'pode ser controlada e geralmente desaparece com o fim do tratamento. A seguir, alguns efeitos adversos mais comumente observados em pacientes oncológicos submetidos à radioterapia em diferentes regiões do corpo: Qualquer área – queda de cabelo e de pelos no local do tratamento (algumas vezes pode ser permanente), irritação cutânea no local da aplicação e fadiga; Cabeça e pescoço – boca seca, espessamento da saliva, dificuldade para engolir, dor de garganta, alteração no sabor dos alimentos,'

In [13]:
docs[0].metadata['source']

'https://grupooncoclinicas.com/tudo-sobre-o-cancer/tratamentos-para-o-cancer/radioterapia'